In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.models import load_model 

from sklearn.model_selection import KFold, StratifiedKFold

import pickle
import time
import numpy as np

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [2]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [3]:
X = np.array(X)
y = np.array(y)

In [4]:
X.shape

(19677, 256, 256, 3)

In [5]:
y.shape

(19677,)

In [6]:
kf = KFold(n_splits = 10, shuffle=True)
                         
skf = StratifiedKFold(n_splits = 10, random_state = 7, shuffle = True)

In [7]:
NAME = "dimentia-simple-CNN-{}".format(int(time.time()))
initializer = 'he_normal'
model = Sequential()

model.add(Conv2D(32, (5, 5), padding='same', input_shape = (256, 256, 3), kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(256, (3, 3), padding='same', kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(512, (3, 3), padding='same', kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(1024, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1024, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(32, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(32, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1024, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary ()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      2432      
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
activation (Activation)      (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0

In [8]:
def lr_schedule(epoch):
    
    lr = 1e-4 #1e-3
    if epoch > 10: # 120
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

fold_no = 1

for train, val in skf.split(X,y): # skf.split

    tensorboard = TensorBoard(log_dir="/home/trojan/Desktop/dimentia/kfold/logs/{}".format(NAME))
    
    lr_scheduler = LearningRateScheduler(lr_schedule)

    #es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
    filepath="model-fold_no{}.h5".format(fold_no)
    #checkpoint = ModelCheckpoint(filepath, monitor='loss', mode='min', verbose=1, save_best_only=True)
    #callbacks_list = checkpoint, es

    #sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(lr=lr_schedule(0)),
                  metrics=['accuracy'],
                  )

    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    model.fit(X[train], y[train],
              batch_size = 8,
              verbose=1,
              epochs=10,
              callbacks=[tensorboard])
    
    # Generate generalization metrics
    scores = model.evaluate(X[val], y[val], verbose=1)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    model.save(filepath)

    # Increase fold number
    fold_no += 1

Learning rate:  0.0001
------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
   1/2214 [..............................] - ETA: 1s - loss: 0.5662 - accuracy: 0.6250WARNING:tensorflow:From /home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
2214/2214 [==============================] - 74s 33ms/step - loss: 0.5501 - accuracy: 0.7085
Epoch 2/10
2214/2214 [==============================] - 74s 33ms/step - loss: 0.3920 - accuracy: 0.8168
Epoch 3/10
2214/2214 [==============================] - 74s 33ms/step - loss: 0.3213 - accuracy: 0.8596
Epoch 4/10
2214/2214 [==============================] - 74s 33ms/step - loss: 0.2635 - accuracy: 0.8950
Epoch 5/10
2214/2214 [==============================] - 74s 33ms/step - loss: 0.2019 - accu

In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')